In [30]:
import torch 
from torch.nn import Embedding
from torch.nn import Linear
import numpy as np

In [31]:
torch.manual_seed(1)

### 1.Embedding

#### 1.1 Embedding的作用

Embedding层的作用是将有限集合中的元素，转变成指定size的向量。这个有限集合可以使NLP中的词汇表，可以使分类任务中的label，当然无论是什么，最终都要以元素索引传递给Embedding。例如，将包含3个元素的词汇表W={'优', '良', '差'}中的每个元素转换为5维向量。如下所示：

In [33]:
# 先定义一个Embedding层：
emb = Embedding(num_embeddings=3, embedding_dim=5)

In [34]:
emb

Embedding(3, 5)

In [35]:
# 转换第一个元素
emb(torch.tensor([0],dtype=torch.int64))

tensor([[ 0.2673, -0.4212, -0.5107, -1.5727, -0.1232]],
       grad_fn=<EmbeddingBackward0>)

In [36]:
# 转换第二个元素
emb(torch.tensor([1],dtype=torch.int64))

tensor([[ 3.5870, -1.8313,  1.5987, -1.2770,  0.3255]],
       grad_fn=<EmbeddingBackward0>)

In [37]:
# 转换第三个元素
emb(torch.tensor([2],dtype=torch.int64))

tensor([[-0.4791,  1.3790,  2.5286,  0.4107, -0.9880]],
       grad_fn=<EmbeddingBackward0>)

In [38]:
# 如果超出词库规模，就会产生异常错误：
# 转换第四个元素
emb(torch.tensor([3],dtype=torch.int64))

IndexError: index out of range in self

初始时，所有向量表示都是随机的，但却并非一成不变的，例如在NLP任务中，随着网络的训练，表示'优'与'良'的两个向量相似度会逐渐减小，而表示'优'与'差'的两个向量相似度会逐渐增大。

#### 1.2 Embedding的用法

接下来我们详细说说pytorch中Embedding层的使用方法。Embedding类主要参数如下：

num_embeddings (int) - 嵌入字典的大小，即共有多少个元素需要转换

embedding_dim (int) - 每个嵌入向量的大小，即转换后获得向量的size

padding_idx (int, optional) - 如果提供的话，输出遇到此下标时用零填充

max_norm (float, optional) - 如果提供的话，会重新归一化词嵌入，使它们的范数小于提供的值

norm_type (float, optional) - 对于max_norm选项计算p范数时的p

scale_grad_by_freq (boolean, optional) - 如果提供的话，会根据字典中单词频率缩放梯度

weight weight (Tensor) -形状为(num_embeddings, embedding_dim)的模块中可学习的权值

 
Embedding是怎么实现的呢？其实，在初始化Embedding层时，Embedding会根据默认随机初始化num_embeddings * embedding_dim的正态分布的权重。以上面例子为例，我们看看它的参数：

In [40]:
# 先定义一个Embedding层：
emb = Embedding(num_embeddings=3, embedding_dim=5)
emb.weight

Parameter containing:
tensor([[-0.9081,  0.5423,  0.1103, -2.2590,  0.6067],
        [-0.1383,  0.8310, -0.2477, -0.8029,  0.2366],
        [ 0.2857,  0.6898, -0.6331,  0.8795, -0.6842]], requires_grad=True)

仔细观察这些权重值，每一行都与上方{'优', '良', '差'}对应。当我们在emb中输入张量torch.tensor([0])时，输出了第一行，当我们在emb中输入张量torch.tensor([1])时，输出了第二行。所以，我们可以猜测，Embedding的工作原理就是初始化一个指定shape的矩阵，在进行转换是，根据输入的tensor值，索引矩阵的行。确实如此，Embedding源码就是这么做的。

当然，Embedding的权重参数也不一定非得随机初始化，也可以手动指定。如下所示，我们先手动初始化一个3 * 5的矩阵，然后将其作为Embedding的权重参数：

In [41]:
# 随机初始化一个3 * 5 的矩阵
emb_weight = torch.rand(3, 5, requires_grad=True)

# 这里需要注意，手动初始化参数时，最好设置requires_grad=True，后续训练时才能更新权重。
emb_weight

tensor([[0.6004, 0.6201, 0.1652, 0.2628, 0.6705],
        [0.5896, 0.2873, 0.3486, 0.9579, 0.4075],
        [0.7819, 0.7165, 0.1768, 0.0748, 0.9799]], requires_grad=True)

In [43]:
# 通过这个预先定义的矩阵，初始化Embedding层
emb2 = Embedding.from_pretrained(emb_weight)

In [45]:
emb2.weight

Parameter containing:
tensor([[0.6004, 0.6201, 0.1652, 0.2628, 0.6705],
        [0.5896, 0.2873, 0.3486, 0.9579, 0.4075],
        [0.7819, 0.7165, 0.1768, 0.0748, 0.9799]])

In [46]:
# 转换第一个元素
emb2(torch.tensor([0],dtype=torch.int64))

tensor([[0.6004, 0.6201, 0.1652, 0.2628, 0.6705]])

In [47]:
# 查看所有权重参数
emb2.weight

Parameter containing:
tensor([[0.6004, 0.6201, 0.1652, 0.2628, 0.6705],
        [0.5896, 0.2873, 0.3486, 0.9579, 0.4075],
        [0.7819, 0.7165, 0.1768, 0.0748, 0.9799]])

这种手动指定参数参数话Embedding层的方式在迁移学习中非常实用，例如在NLP任务中，我们可以使用开源的词向量模型进行初始化，使得我们的模型更快收敛。

### 2 Linear层

#### 2.1 Linear层的作用
Linear层是最古老、最基础的一种网络结构了，作用是对输入向量进行线性变换，输出指定size的向量。例如，将size为3的向量，转为size为5的向量：

In [48]:
# 初始化一个Linear层
lin = Linear(in_features=3, out_features=5)

In [49]:
# 随机初始化一个size为3的向量
x = torch.rand(3)
x

tensor([0.3937, 0.7881, 0.9642])

In [50]:
x.shape

torch.Size([3])

In [51]:
# 经Linear层进行转换
y = lin(x)
y

tensor([ 1.0145,  0.5761,  0.6291, -0.2751, -0.9118], grad_fn=<ViewBackward0>)

In [52]:
y.shape

torch.Size([5])

#### 2.2 Linear的用法
 
Linear类就3个参数：

in_features：指的是输入张量的size
out_features：指的是输出张量的size
bias：是否使用偏置，默认为True，表示使用

In [53]:
w = lin.weight
w

Parameter containing:
tensor([[ 0.0301,  0.3957,  0.1196],
        [ 0.1857,  0.4313,  0.5475],
        [-0.3831,  0.0722,  0.4309],
        [ 0.4183,  0.3587, -0.4178],
        [-0.4158, -0.3492,  0.0725]], requires_grad=True)

In [27]:
b = lin.bias
b

Parameter containing:
tensor([ 0.3505,  0.5120, -0.3236, -0.0950, -0.0112], requires_grad=True)

In [28]:
x.matmul(w.T) + b

tensor([ 0.4185,  0.2063, -0.2675, -0.1059,  0.1010], grad_fn=<AddBackward0>)

In [54]:
# Linear层的参数也可以进行手动修改，不过必须转为Parameter

lin_weight = torch.rand(3, 5, requires_grad=True)
lin_weight

from torch.nn import Parameter
lin.weight = Parameter(lin_weight)

### 3.Embedding与Linear的区别

- Embedding只针对数据集规模有限的离散型数据，Linear即可用于离散型数据，也可用于连续型数据，且对数据集规模无限制。对于Embedding能实现的功能，Liner都能实现，只不过需要先进性一次手动one-hot编码。

- Embedding本质是通过元素的索引，获取矩阵对应行作为输出，而Linear本质是矩阵相乘。